In [324]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

os.chdir(r"C:\Users\siddu\Downloads\US Replication Folder\Trying for India\AIDIS Data\Data Cleaning\22 July 2025\2013")
os.getcwd()

# Filepaths for the datasets
filepaths = {
    'df1': r"C:\Users\siddu\Downloads\US Replication Folder\Trying for India\AIDIS Data\AIDIS 2013 nss 70th round\AIDIS 2013 Visit 1&2 dataset\Visit 1_Block 3_Household Characteristics.dta",
    'df2': r"C:\Users\siddu\Downloads\US Replication Folder\Trying for India\AIDIS Data\AIDIS 2013 nss 70th round\AIDIS 2013 Visit 1&2 dataset\Visit 1_Block 12_Financial assets other than shares and debentures owned.dta",
    'df3': r"C:\Users\siddu\Downloads\US Replication Folder\Trying for India\AIDIS Data\AIDIS 2013 nss 70th round\AIDIS 2013 Visit 1&2 dataset\Visit 1_Block 11_ Shares and debentures owned in co-operative societies & companies.dta",
    'df4': r"C:\Users\siddu\Downloads\US Replication Folder\Trying for India\AIDIS Data\AIDIS 2013 nss 70th round\AIDIS 2013 Visit 1&2 dataset\Visit 1_Block 14_particulars of cash loans payable to institutional, non-institutional agencies.dta",
    'df5': r"C:\Users\siddu\Downloads\US Replication Folder\Trying for India\AIDIS Data\AIDIS 2013 nss 70th round\AIDIS 2013 Visit 1&2 dataset\Visit 1_Block 6_Buildings and other constructions owned by the household as on 30.06.2012.dta",
    'df6': r"C:\Users\siddu\Downloads\US Replication Folder\Trying for India\AIDIS Data\AIDIS 2013 nss 70th round\AIDIS 2013 Visit 1&2 dataset\Visit 1_Block 5pt2_Details of land owned by the household as on 30.06.12.dta",
    'df7': r"C:\Users\siddu\Downloads\US Replication Folder\Trying for India\AIDIS Data\AIDIS 2013 nss 70th round\AIDIS 2013 Visit 1&2 dataset\Visit 1_Block 5pt1_Details of land owned by the household as on 30.06.12.dta",
}

In [325]:
# Load dataframes
dataframes = {name: pd.read_stata(path) for name, path in filepaths.items()}

# Print unique HHIDs in first seven datasets
for i in range(1, 8):
    df = dataframes[f'df{i}']
    unique_hhids = set(df['HHID'].dropna())
    print(f"Unique HHIDs in df{i}: {len(unique_hhids)}")

# Compute common HHIDs: intersection of first five & union of df6, df7
common_hhids = set.intersection(*[
    set(dataframes[f'df{i}']['HHID'].dropna()) for i in range(1, 6)
]) & set.union(*[
    set(dataframes[f'df{i}']['HHID'].dropna()) for i in range(6, 8)
])

print(f"Common HHIDs across: {len(common_hhids)}")

# Filter each dataframe to keep only rows with HHID in common_hhids
for name in dataframes:
    df = dataframes[name]
    df_filtered = df[df['HHID'].isin(common_hhids)].copy()
    dataframes[name] = df_filtered  # Replace with filtered version

# Optional: Print how many rows remain in each
for name, df in dataframes.items():
    print(f"{name}: {len(df)} rows after filtering")

# Reset index
for key in dataframes:
    dataframes[key] = dataframes[key].reset_index(drop=True)

# Convert all columns to numeric except 'HHID' for each dataframe
for i in range(1, 8):
    df = dataframes[f'df{i}']
    cols_to_convert = df.columns.difference(['HHID'])
    df[cols_to_convert] = df[cols_to_convert].apply(pd.to_numeric, errors='coerce')
    dataframes[f'df{i}'] = df  # Update dictionary

# Unpack if needed
df1, df2, df3, df4, df5, df6, df7 = [dataframes[f'df{i}'] for i in range(1, 8)]


Unique HHIDs in df1: 110800
Unique HHIDs in df2: 101412
Unique HHIDs in df3: 3836
Unique HHIDs in df4: 73720
Unique HHIDs in df5: 95488
Unique HHIDs in df6: 34217
Unique HHIDs in df7: 67747
Common HHIDs across: 3000
df1: 3000 rows after filtering
df2: 13313 rows after filtering
df3: 6085 rows after filtering
df4: 9166 rows after filtering
df5: 8636 rows after filtering
df6: 2764 rows after filtering
df7: 8251 rows after filtering


In [326]:
df1.columns

Index(['HHID', 'CentreCodeRndShift', 'Vill_Blk_Slno', 'Round', 'Schedule',
       'Sample', 'Sector', 'State_Region', 'State', 'District_id',
       'State_District', 'Stratum', 'SubStratumNo', 'SubRound', 'SubSample',
       'FODSubRegion', 'HG_SubBlkNo', 'Second_Stratum', 'Hhold_no', 'Visit_no',
       'Level', 'b3q1', 'HH_type', 'b3q3', 'b3q4', 'b3q5', 'b3q6', 'b3q7',
       'b3q8', 'b3q9', 'b3q10', 'b3q11', 'b3q12', 'b3q13', 'b3q14', 'NSS',
       'NSC', 'MLT', 'Weight_SS', 'Weight_SC'],
      dtype='object')

In [327]:
# df1: Keep relevant columns
df1 = df1[['HHID', 'State', 'Weight_SC']].copy()

# df2: Keep all rows for HHIDs that have at least one row with b12_q1 == 11
df2 = df2[['HHID', 'b12_q1', 'b12_q3']].copy()
#hhids_with_11 = df2.loc[df2['b12_q1'] == 11, 'HHID'].unique()
#df2 = df2[df2['HHID'].isin(hhids_with_11)].copy()
df2 = df2[df2['b12_q1'].isin([1,2,3,4,5,6,7])] 

# df3: Keep relevant columns
df3 = df3[['HHID', 'b11_q1', 'b11_q3', 'b11_q6']].copy()
df3 = df3[df3['b11_q1'] == 5] 

# df4: Keep relevant columns
df4 = df4[['HHID', 'b14_q1', 'b14_q6', 'b14_q16', 'b14_q17']].copy()
#hhids_with_99 = df4.loc[df4['b14_q1'] == 99, 'HHID'].unique()
#df4 = df4[df4['HHID'].isin(hhids_with_99)].copy()
df4 = df4[df4['b14_q6'].isin([1,2,3,4,5,6,7,11])]


# df5: Keep only rows where b6_q3 == 11
df5 = df5[['HHID', 'b6_q3', 'b6_q6']].copy()
df5 = df5[df5['b6_q3'] == 11].copy()

# df6: Keep only rows where b5_2_1 == 99
df6 = df6[['HHID', 'b5_2_1', 'b5_2_6']].copy()
df6 = df6[df6['b5_2_1'] == 99].copy()

# df7: Keep only rows where b5_1_1 == 99
df7 = df7[['HHID', 'b5_1_1', 'b5_1_6']].copy()
df7 = df7[df7['b5_1_1'] == 99].copy()

In [328]:
# Renaming, creating, dropping columns 

df1 = df1.rename(columns={
    'Weight_SC': 'Weight'
})

df2  = df2.rename(columns={
    'b12_q1': 'Fin_Asset_Serial',
    'b12_q3': 'Fin_Asset_Value'
})

df3  = df3.rename(columns={
    'b11_q1': 'Share_Asset_serial',
    'b11_q3': 'Share_Asset_val1',
    'b11_q6': 'Share_Asset_val2'
})

df4  = df4.rename(columns={
    'b14_q1': 'Liability_serial',
    'b14_q6': 'Credit_Agency',
    'b14_q16': 'Liability_val1',
    'b14_q17': 'Liability_val2'
})

df5  = df5.rename(columns={
    'b6_q3': 'Building_serial',
    'b6_q6': 'Building_value'
})

df6  = df6.rename(columns={
    'b5_2_1': 'Urban_land_serial',
    'b5_2_6': 'Urban_land_value'
})

df7  = df7.rename(columns={
    'b5_1_1': 'Rural_land_serial',
    'b5_1_6': 'Rural_land_value',
})




In [329]:
df2.columns

Index(['HHID', 'Fin_Asset_Serial', 'Fin_Asset_Value'], dtype='object')

In [330]:
# df2 (Financial Asset value)
df2['Fin_Asset_Value'] = df2['Fin_Asset_Value'].fillna(0)
df2 = df2.reset_index(drop=True)


# df3 (Share Asset value)

df3['Share_Asset_Value'] = df3['Share_Asset_val1'].combine_first(df3['Share_Asset_val2'])
df3['Share_Asset_Value'] = df3['Share_Asset_Value'].fillna(0)

In [331]:
df3.head()

,HHID,Share_Asset_serial,Share_Asset_val1,Share_Asset_val2,Share_Asset_Value
1,247332101,5,700000.0,700000.0,700000.0
2,224701102,5,160000.0,160000.0,160000.0
4,224811101,5,17000.0,17000.0,17000.0
7,221171103,5,100000.0,100000.0,100000.0
8,221531101,5,400.0,400.0,400.0


In [332]:
df4['Credit_Agency'].value_counts()

Credit_Agency
2.0     2673
3.0     1091
11.0     107
7.0       88
1.0       68
6.0       56
4.0       33
5.0        7
Name: count, dtype: int64

In [333]:
# df4 (Liability value)

df4['Liability_value'] = df4['Liability_val1'].combine_first(df4['Liability_val2'])
df4['Liability_value'] = df4['Liability_value'].fillna(0)

In [334]:
# df5 (Buildings value)
df5['Building_value'] = df5['Building_value'].fillna(0)


# df6 (Urban land value)

df6['Urban_land_value'] = df6['Urban_land_value'].fillna(0)


# df7 (Rural land value)

df7['Rural_land_value'] = df7['Rural_land_value'].fillna(0)


# Dropping unnecessary columns

# df2.columns

df3.drop(['Share_Asset_val1', 'Share_Asset_val2'], axis=1, inplace=True)
df4.drop(['Liability_serial', 'Liability_val1', 'Liability_val2'], axis=1, inplace=True)
df5.drop(['Building_serial'], axis=1, inplace=True)
df6.drop(['Urban_land_serial'], axis=1, inplace=True)
df7.drop(['Rural_land_serial'], axis=1, inplace=True)

In [335]:
print(df2['Fin_Asset_Serial'].value_counts(dropna=False))

# Step 2: Group by HHID and sum Fin_Asset_Value
summary_rows = df2.groupby('HHID', as_index=False)['Fin_Asset_Value'].sum()
summary_rows['Fin_Asset_Serial'] = 999

# Step 3: Add other columns with NaN or defaults
for col in df2.columns:
    if col not in ['HHID', 'Fin_Asset_Serial', 'Fin_Asset_Value']:
        summary_rows[col] = np.nan

# Step 4: Concatenate the new rows
df2_mod = pd.concat([df2, summary_rows], ignore_index=True)

# Step 5: Sort
df2_mod = df2_mod.sort_values(by=['HHID', 'Fin_Asset_Serial']).reset_index(drop=True)

# Step 6: Save
df2_mod.to_stata("mergedforu.dta", write_index=False)

print(df2_mod.head())
print(df2_mod['HHID'].nunique())

df2_mod['Fin_Asset_Serial'].value_counts(dropna=False)

# Modify serials and labels, if any

# df2_mod
#df2_mod.loc[df2_mod['Fin_Asset_Serial'].isin([1]), 'Fin_Asset_Serial'] = 31
df2_mod.loc[df2_mod['Fin_Asset_Serial'].isin([2,3,4]), 'Fin_Asset_Serial'] = 32
df2_mod.loc[df2_mod['Fin_Asset_Serial'].isin([7]), 'Fin_Asset_Serial'] = 34
df2_mod.loc[df2_mod['Fin_Asset_Serial'].isin([5,6]), 'Fin_Asset_Serial'] = 35
df2_mod.loc[df2_mod['Fin_Asset_Serial'].isin([1]), 'Fin_Asset_Serial'] = 36
df2_mod.loc[df2_mod['Fin_Asset_Serial'].isin([999]), 'Fin_Asset_Serial'] = 37

df2_mod = df2_mod.reset_index(drop=True)

# Fin Asset Mapping

Fin_Asset_map = {

    31.0: 'Currency',
    32.0: 'Deposits',
    34.0: 'Life Insurance fund',
    35.0: 'Provident and Pension fund',
    36.0: 'Claims on Government',
    37.0: 'Total financial assets (other thanshares and related instruments)'
}

df2_mod = df2_mod.groupby(['HHID', 'Fin_Asset_Serial'], as_index=False).agg({
    'Fin_Asset_Value': 'sum'  
})

df2_mod = df2_mod.reset_index(drop=True)

df2_mod.to_stata("mergedforu.dta", write_index=False)

df2_mod['Fin_Asset_Serial'].value_counts()

df2 = df2_mod.copy()

Fin_Asset_Serial
2    2211
7    1242
1     629
4     490
6     466
3      94
5      35
Name: count, dtype: int64
        HHID  Fin_Asset_Serial  Fin_Asset_Value
0  100131102                 2           3237.0
1  100131102                 7           7910.0
2  100131102               999          11147.0
3  100131105                 2          28600.0
4  100131105                 7          26536.0
2653


In [336]:
df3['Share_Asset_serial'].value_counts()

df3 = df3.reset_index(drop=True)

# Share_Asset_Map

# Mapping dictionary using float keys (since your column is float64)
Share_Asset_Map = {
    5.0: 'Shares and Debentures'
}


In [337]:
df4.head()

,HHID,Credit_Agency,Liability_value
5,247332101,6.0,684760.0
6,224701102,3.0,50000.0
9,224811101,3.0,44000.0
11,221171103,3.0,32100.0
14,221531101,3.0,86339.0


In [338]:
print(df4['Credit_Agency'].value_counts(dropna=False))

Credit_Agency
2.0     2673
3.0     1091
11.0     107
7.0       88
1.0       68
6.0       56
4.0       33
5.0        7
Name: count, dtype: int64


In [339]:
print(df4['Credit_Agency'].value_counts(dropna=False))

# Step 2: Group by HHID and sum Liability_value
summary_rows_liab = df4.groupby('HHID', as_index=False)['Liability_value'].sum()
summary_rows_liab['Credit_Agency'] = 999

# Step 3: Add other columns with NaN or defaults
for col in df4.columns:
    if col not in ['HHID', 'Credit_Agency', 'Liability_value']:
        summary_rows_liab[col] = np.nan

# Step 4: Concatenate the new rows
df4_mod = pd.concat([df4, summary_rows_liab], ignore_index=True)

# Step 5: Sort
df4_mod = df4_mod.sort_values(by=['HHID', 'Credit_Agency']).reset_index(drop=True)

# Step 6: Save
df4_mod.to_stata("mergedforu_liab.dta", write_index=False)

print(df4_mod.head())
print(df4_mod['HHID'].nunique())


# df4_mod
df4_mod.loc[df4_mod['Credit_Agency'].isin([2,3]), 'Credit_Agency'] = 91
df4_mod.loc[df4_mod['Credit_Agency'].isin([1,4,5,6,7,11]), 'Credit_Agency'] = 92

df4_mod = df4_mod.reset_index(drop=True)

# Liability Map

Credit_Agency_Map = {

    91.0: 'Bank advances',
    92.0: 'Non-banking loans and advances',
    999.0: 'Total Institutional Liabilities'
}

df4_mod = df4_mod.groupby(['HHID', 'Credit_Agency'], as_index=False).agg({
    'Liability_value': 'sum'  
})

df4_mod = df4_mod.reset_index(drop=True)

df4_mod.to_stata("mergedforu_liab.dta", write_index=False)

df4_mod['Credit_Agency'].value_counts()

df4 = df4_mod.copy()

Credit_Agency
2.0     2673
3.0     1091
11.0     107
7.0       88
1.0       68
6.0       56
4.0       33
5.0        7
Name: count, dtype: int64
        HHID  Credit_Agency  Liability_value
0  100131102            3.0          50580.0
1  100131102            3.0         145200.0
2  100131102          999.0         195780.0
3  100131105            3.0          32855.0
4  100131105          999.0          32855.0
2596


In [340]:
# Check for uniqueness of subsets
is_unique = not df4.duplicated(subset=['HHID', 'Credit_Agency']).any()
print(is_unique)


True


In [341]:
dataframes['df1'] = df1
dataframes['df2'] = df2
dataframes['df3'] = df3
dataframes['df4'] = df4
dataframes['df5'] = df5
dataframes['df6'] = df6
dataframes['df7'] = df7


In [342]:
# Print unique HHIDs in first seven datasets
for i in range(1, 8):
    df = dataframes[f'df{i}']
    unique_hhids = set(df['HHID'].dropna())
    print(f"Unique HHIDs in df{i}: {len(unique_hhids)}")

# Compute common HHIDs: intersection of first five & union of df6, df7
common_hhids = set.intersection(*[
    set(dataframes[f'df{i}']['HHID'].dropna()) for i in range(1, 6)
]) & set.union(*[
    set(dataframes[f'df{i}']['HHID'].dropna()) for i in range(6, 8)
])

print(f"Common HHIDs across: {len(common_hhids)}")

# Filter each dataframe to keep only rows with HHID in common_hhids
for name in dataframes:
    df = dataframes[name]
    df_filtered = df[df['HHID'].isin(common_hhids)].copy()
    dataframes[name] = df_filtered  # Replace with filtered version

# Optional: Print how many rows remain in each
for name, df in dataframes.items():
    print(f"{name}: {len(df)} rows after filtering")

# Reset index
for key in dataframes:
    dataframes[key] = dataframes[key].reset_index(drop=True)

# Convert all columns to numeric except 'HHID' for each dataframe
for i in range(1, 8):
    df = dataframes[f'df{i}']
    cols_to_convert = df.columns.difference(['HHID'])
    df[cols_to_convert] = df[cols_to_convert].apply(pd.to_numeric, errors='coerce')
    dataframes[f'df{i}'] = df  # Update dictionary

# Unpack if needed
df1, df2, df3, df4, df5, df6, df7 = [dataframes[f'df{i}'] for i in range(1, 8)]

Unique HHIDs in df1: 3000
Unique HHIDs in df2: 2653
Unique HHIDs in df3: 2991
Unique HHIDs in df4: 2596
Unique HHIDs in df5: 3000
Unique HHIDs in df6: 1107
Unique HHIDs in df7: 2166
Common HHIDs across: 2286
df1: 2286 rows after filtering
df2: 6405 rows after filtering
df3: 2286 rows after filtering
df4: 4713 rows after filtering
df5: 2286 rows after filtering
df6: 935 rows after filtering
df7: 1588 rows after filtering


In [343]:
# Merging all datasets
from functools import reduce

dfs = [df1, df2, df3, df4, df5, df6, df7]

merged_df = reduce(lambda left, right: pd.merge(left, right, on='HHID', how='outer'), dfs)

merged_df.to_stata("merged.dta", write_index=False)

unique_hhids = set(merged_df['HHID'].dropna())
print(f"Unique HHIDs: {len(unique_hhids)}")


# Land value
merged_df['Land_value'] = merged_df[['Urban_land_value', 'Rural_land_value']].sum(axis=1, min_count=1).fillna(0)

# Real Estate Value

merged_df['Real Estate'] = merged_df['Building_value'] + merged_df['Land_value']

merged_df.to_stata("merged1.dta", write_index=False)

merged_df1 = merged_df.copy()

merged_df1.columns

merged_df1.drop(['Building_value', 'Urban_land_value',
                'Rural_land_value', 'Land_value'
                ], axis=1, inplace=True)

merged_df1.to_stata("merged1.dta", write_index=False)

merged_df1.columns

# Check for duplicates
group_cols = ['HHID', 'Credit_Agency', 'Fin_Asset_Serial', 'Share_Asset_serial']
dup_check = merged_df1.duplicated(subset=group_cols)
print(f"Number of duplicates: {dup_check.sum()}")

unique_count = merged_df1.loc[merged_df1['Fin_Asset_Serial'] == 37, 'HHID'].nunique()
print(f"Unique HHIDs with Fin_Asset_Serial = 37: {unique_count}")

unique_count = merged_df1.loc[merged_df1['Share_Asset_serial'] == 5, 'HHID'].nunique()
print(f"Unique HHIDs with Share_Asset_serial = 5: {unique_count}")

unique_count = merged_df1.loc[merged_df1['Credit_Agency'] == 999, 'HHID'].nunique()
print(f"Unique HHIDs with Credit_Agency = 999: {unique_count}")


Unique HHIDs: 2286
Number of duplicates: 0
Unique HHIDs with Fin_Asset_Serial = 37: 2286
Unique HHIDs with Share_Asset_serial = 5: 2286
Unique HHIDs with Credit_Agency = 999: 2286


C:\Users\siddu\AppData\Local\Temp\ipykernel_33136\1925433272.py:21: InvalidColumnName: 
Not all pandas column names were valid Stata variable names.
The following replacements have been made:

    Real Estate   ->   Real_Estate

If this is not what you expect, please make sure you have Stata-compliant
column names in your DataFrame (strings only, max 32 characters, only
alphanumerics and underscores, no Stata reserved words)

  merged_df.to_stata("merged1.dta", write_index=False)
C:\Users\siddu\AppData\Local\Temp\ipykernel_33136\1925433272.py:31: InvalidColumnName: 
Not all pandas column names were valid Stata variable names.
The following replacements have been made:

    Real Estate   ->   Real_Estate

If this is not what you expect, please make sure you have Stata-compliant
column names in your DataFrame (strings only, max 32 characters, only
alphanumerics and underscores, no Stata reserved words)

  merged_df1.to_stata("merged1.dta", write_index=False)


In [344]:
cols = ['HHID', 'State','Weight', 'Fin_Asset_Serial', 'Share_Asset_serial',
        'Credit_Agency', 'Share_Asset_Value', 'Fin_Asset_Value',
        'Liability_value', 'Real Estate']

# Create summary dataframe
summary = pd.DataFrame({
    'Column': cols,
    'Missing_Values': [merged_df1[col].isna().sum() for col in cols],
    'Zero_Values': [(merged_df1[col] == 0).sum() for col in cols]
})

print(summary)


# Unique HHIDs

unique_hhids = set(merged_df1['HHID'].dropna())
print(f"Unique HHIDs: {len(unique_hhids)}")

merged_df1.to_stata("merged22.dta", write_index=False)


               Column  Missing_Values  Zero_Values
0                HHID               0            0
1               State               0            0
2              Weight               0            0
3    Fin_Asset_Serial               0            0
4  Share_Asset_serial               0            0
5       Credit_Agency               0            0
6   Share_Asset_Value               0           10
7     Fin_Asset_Value               0            2
8     Liability_value               0           62
9         Real Estate               0            0
Unique HHIDs: 2286


C:\Users\siddu\AppData\Local\Temp\ipykernel_33136\3797293583.py:20: InvalidColumnName: 
Not all pandas column names were valid Stata variable names.
The following replacements have been made:

    Real Estate   ->   Real_Estate

If this is not what you expect, please make sure you have Stata-compliant
column names in your DataFrame (strings only, max 32 characters, only
alphanumerics and underscores, no Stata reserved words)

  merged_df1.to_stata("merged22.dta", write_index=False)


In [345]:
merged_df3 = merged_df1.copy()

# Create the new column
merged_df3['Fin_Asset'] = merged_df3['Fin_Asset_Serial'].map(Fin_Asset_map)
merged_df3['Share_Asset'] = merged_df3['Share_Asset_serial'].map(Share_Asset_Map)
merged_df3['Liability'] = merged_df3['Credit_Agency'].map(Credit_Agency_Map)

# HHIDs satisfying each condition
hhid_fin = set(merged_df3.loc[merged_df3['Fin_Asset_Serial'] == 37, 'HHID'])
hhid_share = set(merged_df3.loc[merged_df3['Share_Asset_serial'] == 5, 'HHID'])
hhid_credit = set(merged_df3.loc[merged_df3['Credit_Agency'] == 999, 'HHID'])

# Intersection: HHIDs present in all three
common_hhids = hhid_fin & hhid_share & hhid_credit

# Keep only those rows
filtered_df = merged_df3[merged_df3['HHID'].isin(common_hhids)].copy()

# Print how many unique HHIDs remain
print(f"Unique HHIDs present in all three groups: {len(common_hhids)}")

#filtered_df.drop(['Fin_Asset', 'Share_Asset', 'Liability'], axis=1, inplace=True)

filtered_df.to_stata("final(1)_13.dta", write_index=False)

Unique HHIDs present in all three groups: 2286


C:\Users\siddu\AppData\Local\Temp\ipykernel_33136\1483304361.py:24: InvalidColumnName: 
Not all pandas column names were valid Stata variable names.
The following replacements have been made:

    Real Estate   ->   Real_Estate

If this is not what you expect, please make sure you have Stata-compliant
column names in your DataFrame (strings only, max 32 characters, only
alphanumerics and underscores, no Stata reserved words)

  filtered_df.to_stata("final(1)_13.dta", write_index=False)


In [346]:
# Sanity checks if grouping worked correctly

print(sorted(filtered_df['Fin_Asset_Serial'].unique()))
#print(filtered_df['Fin_Asset_Serial'].value_counts())

print(sorted(filtered_df['Share_Asset_serial'].unique()))
#print(filtered_df['Share_Asset_serial'].value_counts())

print(sorted(filtered_df['Credit_Agency'].unique()))
#print(filtered_df['Credit_Agency'].value_counts())

[32, 34, 35, 36, 37]
[5]
[91.0, 92.0, 999.0]


In [347]:
df_grouped1 = filtered_df.copy()

# Step 1: Filter the relevant rows satisfying your condition
wealth_rows = df_grouped1[
    (df_grouped1['Credit_Agency'] == 999) &
    (df_grouped1['Fin_Asset_Serial'] == 37) &
    (df_grouped1['Share_Asset_serial'] == 5)
].copy()

# Step 2: Compute wealth for these rows
wealth_rows['Gross wealth'] = (
    wealth_rows['Fin_Asset_Value'] +
    wealth_rows['Share_Asset_Value'] +
    wealth_rows['Real Estate']
)

wealth_rows['Total Financial Assets'] = (
    wealth_rows['Fin_Asset_Value']+
    wealth_rows['Share_Asset_Value']
)

wealth_rows['Total Institutional liabilities'] = wealth_rows['Liability_value']

# Step 3: Keep only HHID and wealth columns
wealth_per_hhid = wealth_rows[['HHID', 'Gross wealth', 'Total Financial Assets', 'Total Institutional liabilities']]

# Step 4: Merge wealth back to the full dataset based on HHID
df_grouped1 = df_grouped1.merge(wealth_per_hhid, on='HHID', how='left')

# Step 5: View result
print(df_grouped1[['HHID', 'Gross wealth']].drop_duplicates().head())

df_grouped1.to_stata("final(25).dta", write_index=False)

         HHID  Gross wealth
0   100131102      324547.0
6   100131105      357936.0
12  100131106      447196.0
18  100141105     2103565.0
28  100141301      600911.0


C:\Users\siddu\AppData\Local\Temp\ipykernel_33136\2434637912.py:33: InvalidColumnName: 
Not all pandas column names were valid Stata variable names.
The following replacements have been made:

    Real Estate   ->   Real_Estate
    Gross wealth   ->   Gross_wealth
    Total Financial Assets   ->   Total_Financial_Assets
    Total Institutional liabilities   ->   Total_Institutional_liabilities

If this is not what you expect, please make sure you have Stata-compliant
column names in your DataFrame (strings only, max 32 characters, only
alphanumerics and underscores, no Stata reserved words)

  df_grouped1.to_stata("final(25).dta", write_index=False)


In [348]:
df_grouped1.columns

Index(['HHID', 'State', 'Weight', 'Fin_Asset_Serial', 'Fin_Asset_Value',
       'Share_Asset_serial', 'Share_Asset_Value', 'Credit_Agency',
       'Liability_value', 'Real Estate', 'Fin_Asset', 'Share_Asset',
       'Liability', 'Gross wealth', 'Total Financial Assets',
       'Total Institutional liabilities'],
      dtype='object')

In [349]:
df_grouped2 =  df_grouped1.copy()

import numpy as np

# Initialize columns with NaN
df_grouped2['Asset_serial'] = np.nan
df_grouped2['Asset_Value'] = np.nan
df_grouped2['Asset_Name'] = np.nan

group_cols = ['HHID', 'Credit_Agency', 'Fin_Asset_Serial', 'Share_Asset_serial']

df_grouped2['HHID'] = df_grouped2['HHID'].astype(str)

#df_grouped2.to_stata("final(26).dta", write_index=False)


df_grouped2.to_stata("final(26).dta", write_index=False)


# Empty list to store new rows
rows = []

# Loop over each unique group
for _, group in df_grouped2.groupby(group_cols):

    # Get first row for copying other columns
    first_row = group.iloc[0].to_dict()

    # 1. Financial Asset Row
    if not np.isnan(first_row['Fin_Asset_Value']):
        fa_row = first_row.copy()
        fa_row['Asset_serial'] = first_row['Fin_Asset_Serial']
        fa_row['Asset_Value'] = first_row['Fin_Asset_Value']
        fa_row['Asset_Name'] = first_row['Fin_Asset']
        rows.append(fa_row)

    # 2. Share Asset Row
    if not np.isnan(first_row['Share_Asset_Value']):
        sa_row = first_row.copy()
        sa_row['Asset_serial'] = 200 + first_row['Share_Asset_serial']  # As you specified
        sa_row['Asset_Value'] = first_row['Share_Asset_Value']
        sa_row['Asset_Name'] = first_row['Share_Asset']
        rows.append(sa_row)

    # 3. Real Estate Row
    if not np.isnan(first_row['Real Estate']):
        re_row = first_row.copy()
        re_row['Asset_serial'] = 300
        re_row['Asset_Value'] = first_row['Real Estate']
        re_row['Asset_Name'] = 'Real Estate'
        rows.append(re_row)

    # 4. Gross wealth Row
    if not np.isnan(first_row['Gross wealth']):
        w_row = first_row.copy()
        w_row['Asset_serial'] = 1000
        w_row['Asset_Value'] = first_row['Gross wealth']
        w_row['Asset_Name'] = 'Gross wealth'
        rows.append(w_row)

    # 6. Total Financial Assets Row
    if not np.isnan(first_row['Total Financial Assets']):
        tfa_row = first_row.copy()
        tfa_row['Asset_serial'] = 1500
        tfa_row['Asset_Value'] = first_row['Total Financial Assets']
        tfa_row['Asset_Name'] = 'Total Financial Assets'
        rows.append(tfa_row)   

# Convert to DataFrame
asset_df = pd.DataFrame(rows)

# Optional: Arrange columns nicely
cols_order = group_cols + ['Asset_serial', 'Asset_Value'] + [col for col in df_grouped2.columns if col not in group_cols + ['Asset_serial', 'Asset_Value']]
asset_df = asset_df[cols_order]

# Check result
print(asset_df.head())



# Renaming Liabilities

asset_df = asset_df.rename(columns={
    'Credit_Agency': 'Liability_serial',
    'Liability': 'Liability_name'
})

asset_df.to_stata("final(28).dta", write_index=False)



C:\Users\siddu\AppData\Local\Temp\ipykernel_33136\1759378772.py:17: InvalidColumnName: 
Not all pandas column names were valid Stata variable names.
The following replacements have been made:

    Real Estate   ->   Real_Estate
    Gross wealth   ->   Gross_wealth
    Total Financial Assets   ->   Total_Financial_Assets
    Total Institutional liabilities   ->   Total_Institutional_liabilities

If this is not what you expect, please make sure you have Stata-compliant
column names in your DataFrame (strings only, max 32 characters, only
alphanumerics and underscores, no Stata reserved words)

  df_grouped2.to_stata("final(26).dta", write_index=False)


        HHID  Credit_Agency  Fin_Asset_Serial  Share_Asset_serial  \
0  100131102           91.0                32                   5   
1  100131102           91.0                32                   5   
2  100131102           91.0                32                   5   
3  100131102           91.0                32                   5   
4  100131102           91.0                32                   5   

   Asset_serial  Asset_Value  State  Weight  Fin_Asset_Value  \
0            32       3237.0     33  3187.5           3237.0   
1           205        600.0     33  3187.5           3237.0   
2           300     312800.0     33  3187.5           3237.0   
3          1000     324547.0     33  3187.5           3237.0   
4          1500      11747.0     33  3187.5           3237.0   

   Share_Asset_Value  Liability_value  Real Estate Fin_Asset  \
0              600.0         195780.0     312800.0  Deposits   
1              600.0         195780.0     312800.0  Deposits   
2       

C:\Users\siddu\AppData\Local\Temp\ipykernel_33136\1759378772.py:88: InvalidColumnName: 
Not all pandas column names were valid Stata variable names.
The following replacements have been made:

    Real Estate   ->   Real_Estate
    Gross wealth   ->   Gross_wealth
    Total Financial Assets   ->   Total_Financial_Assets
    Total Institutional liabilities   ->   Total_Institutional_liabilities

If this is not what you expect, please make sure you have Stata-compliant
column names in your DataFrame (strings only, max 32 characters, only
alphanumerics and underscores, no Stata reserved words)

  asset_df.to_stata("final(28).dta", write_index=False)


In [350]:
group_cols = ['HHID', 'Liability_serial', 'Fin_Asset_Serial', 'Share_Asset_serial']
group_cols1 = ['HHID', 'Liability_serial', 'Fin_Asset_Serial', 'Share_Asset_serial', 'Asset_serial']

# Count unique combinations
num_subsets = asset_df[group_cols].drop_duplicates().shape[0]

print(f"Number of unique subsets: {num_subsets}")

# Check for duplicates
dup_check = asset_df.duplicated(subset=group_cols)
print(f"Number of duplicates: {dup_check.sum()}")

# Check for duplicates
dup_check = asset_df.duplicated(subset=group_cols1)
print(f"Number of duplicates: {dup_check.sum()}")

Number of unique subsets: 13257
Number of duplicates: 53028
Number of duplicates: 0


In [351]:
# Check for missing values

# Check how many missing in each column
print(asset_df[['Asset_serial', 'Asset_Value', 'Asset_Name']].isna().sum())

# Optionally, view rows with missing in any of them
missing_rows = asset_df[
    asset_df[['Asset_serial', 'Asset_Value', 'Asset_Name']].isna().any(axis=1)
]

print(f"\nTotal rows with missing values: {len(missing_rows)}")
print(missing_rows.head())


Asset_serial    0
Asset_Value     0
Asset_Name      0
dtype: int64

Total rows with missing values: 0
Empty DataFrame
Columns: [HHID, Liability_serial, Fin_Asset_Serial, Share_Asset_serial, Asset_serial, Asset_Value, State, Weight, Fin_Asset_Value, Share_Asset_Value, Liability_value, Real Estate, Fin_Asset, Share_Asset, Liability_name, Gross wealth, Total Financial Assets, Total Institutional liabilities, Asset_Name]
Index: []


In [352]:
final_df = asset_df.copy()

final_df.drop(['Fin_Asset_Serial', 
               'Share_Asset_serial', 
               'Fin_Asset_Value',
                'Share_Asset_Value', 
                'Fin_Asset',
                'Share_Asset',               
                ], axis=1, inplace=True)

final_df.to_stata("final(2013).dta", write_index=False)

final_df = final_df[['HHID', 'State' , 'Weight','Gross wealth',  
                    'Liability_serial', 'Liability_name','Liability_value',
                    'Asset_serial', 'Asset_Name', 'Asset_Value',
                    'Real Estate', 'Total Financial Assets', 'Total Institutional liabilities', 
                    ]].copy()

final_df.to_stata("final(2013).dta", write_index=False)

final_df['HHID'].nunique()

final_df1 = final_df[['HHID', 'State', 'Weight', 'Gross wealth',  
                    'Liability_serial', 'Liability_name','Liability_value',
                    'Asset_serial', 'Asset_Name', 'Asset_Value'
                    ]].copy()

final_df1.to_stata("final(2013)_1.dta", write_index=False)
final_df1['HHID'].nunique()

C:\Users\siddu\AppData\Local\Temp\ipykernel_33136\3884663063.py:11: InvalidColumnName: 
Not all pandas column names were valid Stata variable names.
The following replacements have been made:

    Real Estate   ->   Real_Estate
    Gross wealth   ->   Gross_wealth
    Total Financial Assets   ->   Total_Financial_Assets
    Total Institutional liabilities   ->   Total_Institutional_liabilities

If this is not what you expect, please make sure you have Stata-compliant
column names in your DataFrame (strings only, max 32 characters, only
alphanumerics and underscores, no Stata reserved words)

  final_df.to_stata("final(2013).dta", write_index=False)
C:\Users\siddu\AppData\Local\Temp\ipykernel_33136\3884663063.py:19: InvalidColumnName: 
Not all pandas column names were valid Stata variable names.
The following replacements have been made:

    Gross wealth   ->   Gross_wealth
    Real Estate   ->   Real_Estate
    Total Financial Assets   ->   Total_Financial_Assets
    Total Institution

2286

In [353]:
print(sorted(final_df1['Asset_serial'].unique()))

[32, 34, 35, 36, 37, 205, 300, 1000, 1500]


In [354]:
group_cols1 = ['HHID', 'Liability_serial', 'Asset_serial']

final_df2 = final_df1.drop_duplicates(subset=group_cols1, keep='first').reset_index(drop=True)

final_df2.to_stata("final(2013)_2.dta", write_index=False)

# Check for repetition

# Check for uniqueness 

duplicates = final_df2.duplicated(subset=['Asset_serial', 'HHID', 'Liability_serial'], keep=False)

# Check if there are any duplicates
if duplicates.any():
    print("Some (Asset_serial, HHID) combinations are repeated:")
    print(final_df2.loc[duplicates, ['Asset_serial', 'HHID']].sort_values(['Asset_serial', 'HHID']))
else:
    print(" All HHIDs are unique within each Asset_serial category.")

# Check if there are any duplicates
if duplicates.any():
    print("Some (Liability_serial, HHID) combinations are repeated:")
    print(final_df2.loc[duplicates, ['Liability_serial', 'HHID']].sort_values(['Liability_serial', 'HHID']))
else:
    print(" All HHIDs are unique within each Liability_serial category.")


 All HHIDs are unique within each Asset_serial category.
 All HHIDs are unique within each Liability_serial category.


C:\Users\siddu\AppData\Local\Temp\ipykernel_33136\1595470577.py:5: InvalidColumnName: 
Not all pandas column names were valid Stata variable names.
The following replacements have been made:

    Gross wealth   ->   Gross_wealth

If this is not what you expect, please make sure you have Stata-compliant
column names in your DataFrame (strings only, max 32 characters, only
alphanumerics and underscores, no Stata reserved words)

  final_df2.to_stata("final(2013)_2.dta", write_index=False)


In [355]:
final_df2['HHID'].nunique()

2286

# Add State names corresponding to their codes

In [356]:
# final_df2 (Merge D&D, D&N)
final_df2.loc[final_df2['State'].isin([25,26]), 'State'] = 25
final_df2['State'].value_counts()

# State Map

State_map = {
    1.0: 'Jammu and Kashmir',
    2.0: 'Himachal Pradesh',
    3.0: 'Punjab',
    4.0: 'Chandigarh',
    5.0: 'Uttarakhand',
    6.0: 'Haryana',
    7.0: 'Delhi',
    8.0: 'Rajasthan',
    9.0: 'Uttar Pradesh',
    10.0: 'Bihar',
    11.0: 'Sikkim',
    12.0: 'Arunachal Pradesh',
    13.0: 'Nagaland',
    14.0: 'Manipur',
    15.0: 'Mizoram',
    16.0: 'Tripura',
    17.0: 'Meghalaya',
    18.0: 'Assam',
    19.0: 'West Bengal',
    20.0: 'Jharkhand',
    21.0: 'Odisha',
    22.0: 'Chhattishgarh',
    23.0: 'Madhya Pradesh',
    24.0: 'Gujarat',
    25.0: 'Daman and Diu and Dadra and Nagar Haveli',
    27.0: 'Maharashtra',
    28.0: 'Andhra Pradesh',
    29.0: 'Karnataka',
    30.0: 'Goa',
    31.0: 'Lakshadweep',
    32.0: 'Kerala',
    33.0: 'Tamilnadu',
    34.0: 'Puducherry',
    35.0: 'Andaman & Nicobar',
    36.0: 'Telengana'
}

final_df3 = final_df2.copy()

final_df3['State_name'] = final_df3['State'].map(State_map)

final_df3.to_stata('2013_final.dta', write_index=False)

C:\Users\siddu\AppData\Local\Temp\ipykernel_33136\1197275976.py:49: InvalidColumnName: 
Not all pandas column names were valid Stata variable names.
The following replacements have been made:

    Gross wealth   ->   Gross_wealth

If this is not what you expect, please make sure you have Stata-compliant
column names in your DataFrame (strings only, max 32 characters, only
alphanumerics and underscores, no Stata reserved words)

  final_df3.to_stata('2013_final.dta', write_index=False)
